In [1]:
import pandas as pd

from bs4 import BeautifulSoup
import requests
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)

C:\Users\Isabel\AppData\Local\Temp\ipykernel_14428\253687804.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## CÓDIGO ROTTEN

In [ ]:
busqueda = # lista de listas con películas

In [23]:
lista_notas = []
errores = []

In [21]:
driver = webdriver.Chrome()
driver.get('https://www.google.com/')
driver.maximize_window()
sleep(2)

driver.find_element('css selector', '#W0wltc > div').click()
sleep(1)


for pelicula in busqueda:
    notas = [f'{pelicula[0]}',]
        
    try:
        driver.find_element('css selector', '#APjFqb').send_keys(pelicula[2],' ', pelicula[4], ' rotten tomatoes', Keys.ENTER)
        sleep(1)

        nota = driver.find_element('css selector', '#rso > div:nth-child(1) > div > div > div:nth-child(3) > div > span:nth-child(2)').text
        
    except:
        errores.append(pelicula)
        
    notas.append(nota[-3:])    
    driver.find_element('css selector', '#APjFqb').clear() 
    
    lista_notas.append(tuple(notas))

In [ ]:
df = lista_notas
df.to_csv('NotasRotten.csv', index=False)

## CÓDIGO IMDB

In [ ]:
info_por_peli = []
fallos = []

In [ ]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))


# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]

    try:
    # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
            
    except:
        fallos.append(peli)



In [ ]:
lista_tuplas = []

In [ ]:
# recogida de los datos
for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    patron_duracion = re.compile(r'\b(?:\d+h\s*)?\d+min\b')
    tupla_datos = [peli[0],] 

    # listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            
            if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

    # añadimos a la tupla los datos de la lista intermedia
    for elemento in [direccion, guion, nota, duracion, argumento]:
        if elemento != []:
            tupla_datos.append(elemento[0])
        else:
            tupla_datos.append(np.nan)
    
    lista_tuplas.append(tupla_datos)